# XTK: LLM-Powered Explanations Tutorial

This notebook demonstrates how to use XTK's new **LLM-powered explanation** features to generate natural language explanations for symbolic computation steps.

## Features Covered

1. **Rich Rules with Metadata** - Rules can include names, descriptions, categories, and examples
2. **LLM Integration** - Support for Anthropic Claude, OpenAI GPT, and local Ollama
3. **Explanation Generation** - Generate pedagogical explanations for rewrite steps
4. **Fallback Mode** - Works without LLM using structured descriptions
5. **REPL Commands** - `/explain` and `/trace-explain` for interactive use

## Setup

In [ ]:
import sys
sys.path.insert(0, '../src')

from xtk import (
    Expression,
    rewriter,
    RichRule,
    normalize_rules,
    RewriteExplainer
)

print("✅ XTK loaded successfully!")

## Part 1: Rich Rules with Metadata

### Simple Format (Still Works)

The traditional format still works:

In [ ]:
# Simple [pattern, skeleton] format
simple_rule = [["+", ["?v", "x"], 0], [":", "x"]]
print("Simple rule:", simple_rule)

### Rich Format with Metadata

But now you can also use the rich format with metadata:

In [ ]:
# Rich rule format with metadata
add_zero_rule = {
    "pattern": ["+", ["?v", "x"], 0],
    "skeleton": [":", "x"],
    "name": "add-zero-right",
    "description": "Adding zero to any expression doesn't change it (additive identity)",
    "category": "algebra-identity",
    "examples": ["x + 0 = x", "5 + 0 = 5"]
}

# Create RichRule object
rich_rule = RichRule.from_rule(add_zero_rule)
print(f"Rule name: {rich_rule.name}")
print(f"Description: {rich_rule.description}")
print(f"Category: {rich_rule.category}")
print(f"Examples: {rich_rule.examples}")

### Loading Rich Rules

Let's load some rich rules from the examples:

In [ ]:
# Load rich algebra rules
from xtk.rules.algebra_rules_rich import algebra_rules_rich

# Normalize to both formats
rule_pairs, rich_rules = normalize_rules(algebra_rules_rich)

print(f"Loaded {len(rich_rules)} rich algebra rules:\n")
for i, rule in enumerate(rich_rules[:5]):
    print(f"{i+1}. {rule.name}: {rule.description}")

## Part 2: Using Rules for Rewriting

Rich rules work exactly like regular rules for rewriting:

In [ ]:
# Create rewriter
rewrite_fn = rewriter(rule_pairs)

# Test rewriting
expr = Expression(["+", "x", 0])
print(f"Original: {expr.to_string()}")

result = rewrite_fn(expr.expr)
print(f"Result: {Expression(result).to_string()}")

# Which rule was used?
used_rule = rich_rules[0]  # We know it's the first one
print(f"\nRule used: {used_rule.name}")
print(f"Description: {used_rule.description}")

## Part 3: Explanation Generation (Fallback Mode)

### Without LLM

First, let's use the explainer in fallback mode (no LLM):

In [ ]:
# Create explainer without LLM
explainer = RewriteExplainer.from_config("none")

# Generate explanation
explanation = explainer.explain_step(
    expression="(+ x 0)",
    result="x",
    rule_name="add-zero-right",
    rule_description="Adding zero to any expression doesn't change it (additive identity)"
)

print("Explanation (fallback mode):")
print(explanation)

## Part 4: LLM-Powered Explanations

### Setting Up LLM Provider

To use LLM-powered explanations, you need to set up an API key.

**Option 1: Anthropic Claude**
```python
import os
os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'
explainer = RewriteExplainer.from_config("anthropic")
```

**Option 2: OpenAI GPT**
```python
os.environ['OPENAI_API_KEY'] = 'sk-...'
explainer = RewriteExplainer.from_config("openai", model="gpt-4")
```

**Option 3: Ollama (Local)**
```python
# No API key needed - just run: ollama serve
explainer = RewriteExplainer.from_config("ollama", model="llama2")
```

### Example with Mock LLM

For this demo, we'll use fallback mode, but here's what an LLM explanation would look like:

In [ ]:
# Simulate LLM explanation
llm_explanation = """
The power rule applies here because we're differentiating a power function x².
This fundamental calculus rule states that when you differentiate x^n with respect
to x, you bring down the exponent n as a coefficient and reduce the exponent by 1.
In this case, n=2, so we get 2·x^(2-1) = 2·x^1 = 2x.
""".strip()

print("LLM Explanation Example:")
print(llm_explanation)

## Part 5: Derivative Example

Let's work through a calculus example with explanations:

In [ ]:
# Load derivative rules
from xtk.rules.deriv_rules_rich import deriv_rules_rich

# Normalize rules
deriv_pairs, deriv_rich = normalize_rules(deriv_rules_rich)

print(f"Loaded {len(deriv_rich)} derivative rules:\n")
for i, rule in enumerate(deriv_rich[:5]):
    print(f"{i+1}. {rule.name}")
    print(f"   {rule.description}")
    if rule.examples:
        print(f"   Examples: {', '.join(rule.examples)}")
    print()

In [ ]:
# Differentiate x^2
expr = Expression(["dd", ["^", "x", 2], "x"])
print(f"Expression: {expr.to_string()}")
print("(This means: take the derivative of x² with respect to x)\n")

# Apply rewriting
deriv_rewriter = rewriter(deriv_pairs)
result = deriv_rewriter(expr.expr)
print(f"Result: {Expression(result).to_string()}")

# Generate explanation
power_rule = deriv_rich[8]  # The power rule
explanation = explainer.explain_step(
    expression=expr.to_string(),
    result=Expression(result).to_string(),
    rule_name=power_rule.name,
    rule_description=power_rule.description,
    bindings=[["x", "x"], ["n", 2]]
)

print(f"\nExplanation:")
print(explanation)

## Part 6: Using in the REPL

In the interactive REPL, you can use these commands:

### `/explain` Command

```bash
xtk> /rules load src/xtk/rules/deriv_rules_rich.py
Loaded 20 rules

xtk> (dd (^ x 2) x)
$[0] (dd (^ x 2) x)

xtk> /rw
Rewritten: (* 2 (^ x 1))

xtk> /explain
╭────────────── Rewrite Explanation ──────────────╮
│ Expression: (dd (^ x 2) x)                      │
│ Result: (* 2 (^ x 1))                           │
│ Rule: power-rule                                 │
╰─────────────────────────────────────────────────╯

╭───────────── 📖 Explanation ─────────────╮
│ Applied power-rule: The power rule for   │
│ derivatives: d/dx(x^n) = n·x^(n-1)       │
╰──────────────────────────────────────────╯
```

### `/trace-explain` Command

Shows full trace with explanations for each step:

```bash
xtk> (dd (+ (^ x 2) x) x)
$[0] (dd (+ (^ x 2) x) x)

xtk> /trace-explain
╭──── Rewriting Trace with Explanations ────╮

Initial: (dd (+ (^ x 2) x) x)

Step 1: (+ (dd (^ x 2) x) (dd x x))
  Rule: sum-rule
  💡 The sum rule: the derivative of a sum is the sum of derivatives

Step 2: (+ (* 2 (^ x 1)) 1)
  Rule: power-rule, derivative-of-variable
  💡 Applied power rule and variable rule

Final: (+ (* 2 x) 1)
```

## Part 7: Educational Applications

### Example: Teaching the Product Rule

Let's demonstrate how explanations help teach the product rule:

In [ ]:
# Product rule example: d/dx(x · sin(x))
product_expr = Expression(["dd", ["*", "x", ["sin", "x"]], "x"])
print(f"Problem: Find {product_expr.to_string()}")
print("(derivative of x·sin(x))\n")

# Find the product rule
product_rule = next(r for r in deriv_rich if r.name == "product-rule")

print("Rule Information:")
print(f"Name: {product_rule.name}")
print(f"Description: {product_rule.description}")
print(f"Category: {product_rule.category}")
print(f"Examples: {product_rule.examples}")

# The explanation would guide the student through:
print("\nStep-by-step explanation:")
print("1. Identify f = x and g = sin(x)")
print("2. Find f' = 1 and g' = cos(x)")
print("3. Apply formula: f'·g + f·g' = 1·sin(x) + x·cos(x)")
print("4. Simplify: sin(x) + x·cos(x)")

## Part 8: Benefits Summary

### For Learning
- **Understand WHY** transformations happen
- **See the pattern** in mathematical rules
- **Get immediate feedback** on each step

### For Debugging
- **Track which rules** are being applied
- **Identify unexpected** transformations
- **Understand rule interactions**

### For Documentation
- **Rules self-document** with descriptions
- **Examples embedded** in code
- **Categories organize** knowledge

### Flexibility
- **Works offline** with fallback mode
- **Choose your LLM** (Claude, GPT, Ollama)
- **Caching minimizes** API costs
- **Backwards compatible** with simple rules

## Conclusion

XTK's LLM-powered explanations transform it from a computation tool into an **educational platform**.

By combining:
- **Structured metadata** (names, descriptions, examples)
- **LLM generation** (natural, pedagogical explanations)
- **Interactive REPL** (/explain, /trace-explain commands)

XTK enables users to not just *compute* symbolic expressions, but truly *understand* them.

### Next Steps

1. Try the REPL commands with rich rules
2. Set up an LLM provider for better explanations  
3. Create your own rich rules with metadata
4. Build educational materials using XTK

Happy learning! 🎓